In [1]:
import pandas as pd
from mod.config import DATABASE, EVENT_SHEET, MEMBER_SHEET
from mod import general as gr
from mod import add_member as add

In [ ]:
# 讀取會員表
df_member = gr.GET_DF_FROM_DB(sheet=MEMBER_SHEET)

# 輸入新增會員資料
member_list = add.build_member()

# 將新增資料轉成df
df_temp = pd.DataFrame(member_list)

# 檢查是否有已經加入過會員的資料
df_temp_clean = add.check_and_remove_duplicates(df_member=df_member, df_temp=df_temp)

# 與主表合併
df_member = pd.concat([df_member, df_temp], ignore_index=True)

# 存檔
gr.SAVE_TO_SHEET(df=df_member, sheet=MEMBER_SHEET)


會員資料新增完成！


In [3]:
df_member

,Unnamed: 0,會員姓名,Email,生日,電話,加入日期,加入時間
0,0.0,qaz,qaz@wsx.com,1992-01-01,988555666,2025-12-03,15:24:28
1,1.0,wsx,edc@rfv.com,2005-10-11,988333222,2025-12-03,15:24:50
2,NaN,qaz,qaz@wsx.com,1920-10-01,0988555666,2025-12-03,15:32:19


In [4]:
# PermissionError